In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
from webscrape import Webscrape

In [2]:
#datasets
movies = pd.read_csv("../../data/ml-latest-small/movies.csv")
links = pd.read_csv("../../data/ml-latest-small/links.csv")

In [3]:
# string s, return list of tags
def get_tags(s) : 
    return s.strip().split("|")

In [4]:
# adding colums, binary 0,1 for if the tag exists
all_tags = {
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
#     "(no genres listed)" # not needed - just all 0's
}

for t in all_tags :
    movies[f'Genre_{t}'] = False # default initialize 0


In [5]:
# assignment for genre tags
for index, row in movies.iterrows():
    for x in get_tags(row['genres']):
        if x in all_tags:
            movies.loc[index, f'Genre_{x}'] = True

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
def fix_imdb_string(i):
    s = str(i)
    if len(s) > 7: return s
    if len(s) == 7: return 'tt' + s
    i = len(s)
    return 'tt' + '0' * (7 - i) + s

In [9]:
# test above function
a = [
    1234567,
    123456,
    1234,
    234,
    '012345',
    '0012345'
]
for x in a:
    print(fix_imdb_string(x))

tt1234567
tt0123456
tt0001234
tt0000234
tt0012345
tt0012345


In [44]:
num_scraped = 9742
# filename = f'output{num_scraped}.csv'
filename = 'data3_31.csv'
df = pd.read_csv(filename)
df.drop_duplicates(inplace=True)

# s = set(df['id'])

In [14]:
'''
assignment for webscraped values:
year, imdb rating, audience, runtime
'''
di = {}
scraper = Webscrape()
misses = 0
for index, row in movies.iterrows():
    if misses > 10: break
    imdb_id = fix_imdb_string(links.loc[index, 'imdbId'])

    if imdb_id in s: 
        continue
    try:
        year, imdb_rating, audience, runtime = scraper.get_info(imdb_id[2:]) # without 'tt'
        di[imdb_id] = (year, imdb_rating, audience, runtime)
    except:
        misses += 1
        print(index, imdb_id)

585 tt0118114
665 tt0110693
5620 tt0169590
7313 tt0884762
7581 tt0078588
7600 tt1347439
8128 tt0427531
9603 tt6254796
9726 tt7620650


In [21]:
'''printing stuff one for dict, one for df'''

import csv
############################################################
df.to_csv(f"output{df['id'].size}.csv", index=False)


############################################################
w = csv.writer(open(f"output{len(di)}.csv", "w"))

# loop over dictionary keys and values
for key, val in di.items():

    # write every key and value to file
    w.writerow([key, val])

In [56]:
# dead imdb sites or other problems (tv shows mainly)

imdb_list= [
    118114,
    110693,
    169590,
    884762,
    78588,
    1347439,
    427531,
    6254796,
    7620650
]

mov_lists = []

for x in imdb_list:
    mov_lists.append(links.loc[links['imdbId'] == x, 'movieId'].values[0])

print(mov_lists)

[720, 875, 27155, 77177, 86237, 86668, 101423, 176329, 190209]


In [59]:
for x in mov_lists:
    print(movies.loc[movies['movieId'] == x, 'title'].values[0])

Wallace & Gromit: The Best of Aardman Animation (1996)
Nothing to Lose (1994)
Batman/Superman Movie, The (1998)
Wild China (2008)
Connections (1978)
Louis Theroux: Law & Disorder (2008)
Mezzo Forte (1998)
Ari Shaffir: Double Negative (2017)
Jeff Ross Roasts the Border (2017)


In [60]:
# id: (year, rating, audience, runtime)
helper_di = {
    'tt0118114': ('1996', '8.3', 'G', 90),
    'tt0110693': ('1994', '5.8', 'R', 85),
    'tt0169590': ('1998', '7.8', 'Unrated', 64),
    'tt0884762': ('2008', '8.4', 'Unrated', 60),
    'tt0078588': ('1978', '9.3', 'Unrated', 90),
    'tt1347439': ('2008', '7.4', 'TV-MA' , 120),
    'tt0427531': ('1998', '6.7', 'Unrated', 80),
    'tt6254796': ('2017', '6.8', 'Unrated', 90),
    'tt7620650': ('2017', '6.7', 'Unrated', 90)
}

In [25]:
for ind, row in df.iterrows():
    imdb_id = int(row['id'][2:])
    mov_id = links.loc[links['imdbId'] == imdb_id, 'movieId'].values[0]
    year, rating, audience, runtime = row['val'][1:-1].split(', ')

    movies.loc[movies['movieId'] == mov_id, 'year'] = int(year[1:-1])
    movies.loc[movies['movieId'] == mov_id, 'rating'] = float(rating[1:-1])
    movies.loc[movies['movieId'] == mov_id, 'audience'] = audience
    movies.loc[movies['movieId'] == mov_id, 'runtime'] = int(runtime)
    movies.loc[movies['movieId'] == mov_id, 'imdbId'] = imdb_id


In [26]:
movies.to_csv(f"data3_31.csv", index=False)

In [7]:
utility_matrix = pd.read_csv('../../data/user_rating_matrix.csv', index_col=0)
utility_matrix.columns.name = utility_matrix.index.name
utility_matrix.index.name = 'title'
utility_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
utility_matrix.index == "'71 (2014)"

array([ True, False, False, ..., False, False, False])

In [30]:
utility_matrix.loc[["'71 (2014)"]]

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [89]:
di = {}
li = []
for i, (ind, row) in enumerate (utility_matrix.iterrows()):
    df.loc[df['title'] == ind, 'cleaned_mov_id'] = i

In [90]:
df.head()

,movieId,title,Genre_Thriller,Genre_Sci-Fi,Genre_Mystery,Genre_Documentary,Genre_Western,Genre_Comedy,Genre_Adventure,Genre_Film-Noir,...,Genre_War,Genre_Romance,Genre_Animation,Genre_Children's,year,rating,audience,runtime,imdbId,cleaned_mov_id
0,1,Toy Story (1995),False,False,False,False,False,True,True,False,...,False,False,True,False,1995.0,8.3,'G',81.0,114709.0,8871.0
1,2,Jumanji (1995),False,False,False,False,False,False,True,False,...,False,False,False,False,1995.0,7.0,'PG',104.0,113497.0,4645.0
2,3,Grumpier Old Men (1995),False,False,False,False,False,True,False,False,...,False,True,False,False,1995.0,6.6,'PG-13',101.0,113228.0,3661.0
3,4,Waiting to Exhale (1995),False,False,False,False,False,True,False,False,...,False,True,False,False,1995.0,5.9,'R',124.0,114885.0,9225.0
4,5,Father of the Bride Part II (1995),False,False,False,False,False,True,False,False,...,False,False,False,False,1995.0,6.0,'PG',106.0,113041.0,2967.0


In [99]:
df.to_csv(f"data4_01.csv", index=False)